In [1]:
!pip install git-python
!pip install sacrebleu
!pip install rouge_score
!pip install farasapy
!pip install pyarabic
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 181 kB 35.6 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 116 kB 35.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=974286aafcb76c88e27b7bab9f751621afc33088fbaafae0c9631e2ae653d436
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████████████████████████████

In [2]:
!git clone https://github.com/aub-mind/arabert
!git clone https://github.com/tareknaous/dialectal-conv/


Cloning into 'arabert'...
remote: Enumerating objects: 595, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 595 (delta 37), reused 43 (delta 29), pack-reused 535
Receiving objects: 100% (595/595), 9.14 MiB | 18.68 MiB/s, done.
Resolving deltas: 100% (338/338), done.
Cloning into 'dialectal-conv'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [3]:
!wget https://raw.githubusercontent.com/aub-mind/Arabic-Empathetic-Chatbot/master/arabic-empathetic-conversations.csv

--2022-09-03 17:36:34--  https://raw.githubusercontent.com/aub-mind/Arabic-Empathetic-Chatbot/master/arabic-empathetic-conversations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7873052 (7.5M) [text/plain]
Saving to: ‘arabic-empathetic-conversations.csv’

arabic-empathetic-c 100%[===================>]   7.51M  --.-KB/s    in 0.02s   

2022-09-03 17:36:35 (374 MB/s) - ‘arabic-empathetic-conversations.csv’ saved [7873052/7873052]



In [4]:
import os
import numpy as np
import pandas as pd
import math
from datasets import load_dataset
import transformers
from transformers import BertTokenizer, EncoderDecoderModel
from sacrebleu import corpus_bleu
from transformers import BertTokenizerFast
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [5]:
encoder_max_length = 75
decoder_max_length = 75

model_name = "aubmindlab/bert-base-arabert"

tokenizer = BertTokenizerFast.from_pretrained(model_name)

In [7]:
all_data = load_dataset("ArabicEmpatheticChatbot.py")

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


100%|██████████| 241M/241M [00:15<00:00, 15.4MiB/s]


[2022-09-03 17:37:49,679 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


Generating train split: 0 examples [00:00, ? examples/s]

Dataset arabic_empathetic_chatbot downloaded and prepared to /root/.cache/huggingface/datasets/arabic_empathetic_chatbot/arabic_emp_conv/1.0.0/af81e5e61abf371a71d8d27b4483bf0141f8b51c942075e3fed5df7b73769b6d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
train_data = all_data['train'].train_test_split(test_size=0.1,seed=42)['train']
val_test_data = all_data["train"].train_test_split(test_size=0.1, seed=42)["test"]
val_data = val_test_data.train_test_split(test_size=0.5, seed=42)["train"]
test_data = val_test_data.train_test_split(test_size=0.5, seed=42)["test"]

In [9]:
len_train_data = len(train_data)
len_val_data = len(val_data)
len_test_data = len(test_data)

print("Length of train data ", len_train_data)
print("Length of val data ", len_val_data)
print("Length of test data ", len_test_data)

Length of train data  32965
Length of val data  1831
Length of test data  1832


In [10]:
def process_data_to_model_inputs(batch):
  inputs = tokenizer(batch["context"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["response"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["labels"] = outputs.input_ids.copy()

  batch["labels"] = [
      [-100 if token == tokenizer.pad_token_id else token 
       for token in labels] for labels in batch["labels"]
  ]

  batch["decoder_attention_mask"] = outputs.attention_mask
  return batch

In [11]:
batch_size = 16

In [12]:
train_data = train_data.map(process_data_to_model_inputs,
                            batched=True,
                            batch_size=batch_size,
                            remove_columns=["context", "response"])

train_data.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "decoder_input_ids", 
             "decoder_attention_mask", "labels"]
)

val_data = val_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["context", "response"]
)

val_data.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "decoder_input_ids", 
             "decoder_attention_mask", "labels"]
)

test_data = test_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["context", "response"]
)

test_data.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "decoder_input_ids", 
             "decoder_attention_mask", "labels"]
)

  0%|          | 0/2061 [00:00<?, ?ba/s]

  0%|          | 0/115 [00:00<?, ?ba/s]

  0%|          | 0/115 [00:00<?, ?ba/s]

In [13]:
model_name = "aubmindlab/bert-base-arabert"

arabert2arabert = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name, tie_encoder_decoder=False)

Some weights of the model checkpoint at aubmindlab/bert-base-arabert were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at aubmindlab/bert-base-arabert were not used when initializing BertLMHeadModel: ['cls.seq_relationship.wei

In [14]:
#set special tokens

arabert2arabert.config.decoder_start_token_id = tokenizer.cls_token_id
arabert2arabert.config.eos_token_id = tokenizer.sep_token_id
arabert2arabert.config.pad_token_id = tokenizer.pad_token_id

In [15]:
#sensible parameters for beam search
#set decoding params  

arabert2arabert.config.max_length = 64
arabert2arabert.config.early_stopping = True

arabert2arabert.config.num_beams = 1
arabert2arabert.config.vocab_size = arabert2arabert.config.encoder.vocab_size

In [25]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
        )
    
    sortish_sampler: bool = field(
        default=False, metadata={"help": "Whether to SortishSamler or not."}
        )
    
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
        )
    
    adafactor: bool = field(
        default=False, metadata={"help": "whether to use adafactor"}
        )
    
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
        )
    
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
        )
    
    dropout: Optional[float] = field(
        default=None, metadata={"help": "Dropout probability. Goes into model.config."}
        )
    
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
        )

    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
        )
    
    generation_max_length: Optional[int] = field(
        default=None, metadata={ "help": "The `max_length` to use on each evaluation loop when `predict_with_generate=True`. Will default to the `max_length` value of the model configuration."}
        )
    
    generation_num_beams: Optional[int] = field(
        default=None, metadata={"help": "The `num_beams` to use on each evaluation loop when `predict_with_generate=True`. Will default to the `num_beams` value of the model configuration."},
    )

In [30]:
import torch
import torch.nn as nn

def compute_metrics(pred):
  labels_ids = pred.label_ids
  pred_ids = pred.predictions

  # all unnecessary tokens are removed
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  return{"bleu": round(corpus_bleu(pred_str, [label_str]).score, 4)}

In [31]:
#Set training arguments 
training_args = Seq2SeqTrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    gradient_accumulation_steps=2,
    predict_with_generate=True,
    do_eval=True,
    evaluation_strategy="epoch",
    do_train=True,
    logging_steps=500,  
    save_steps=len_train_data // ( batch_size * 2),  
    warmup_steps=100,
    eval_steps=10,
    #num_train_epochs=5,
    overwrite_output_dir=True,
    save_total_limit=0,
    fp16=True, 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
# instantiate trainer
trainer = transformers.Seq2SeqTrainer(
    model=arabert2arabert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data
)

Using cuda_amp half precision backend


In [33]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: emotion. If emotion are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 32965
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 5150


Epoch,Training Loss,Validation Loss,Bleu
0,0.000400,0.000345,0.000000
1,0.000100,0.000370,0.000000
2,0.000300,0.000471,0.000000
3,0.000000,0.000417,0.000000
4,0.000000,0.000400,0.000000


Saving model checkpoint to ./model/checkpoint-1030
Configuration saved in ./model/checkpoint-1030/config.json
Model weights saved in ./model/checkpoint-1030/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: emotion. If emotion are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num e

TrainOutput(global_step=5150, training_loss=0.00011733515918236769, metrics={'train_runtime': 4292.044, 'train_samples_per_second': 38.402, 'train_steps_per_second': 1.2, 'total_flos': 1.4813445652992e+16, 'train_loss': 0.00011733515918236769, 'epoch': 5.0})

In [65]:
eval_output = trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: emotion. If emotion are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1831
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


In [66]:
perplexity = math.exp(eval_output["eval_loss"])
print("Evaluate Perplexity: {:10,.2f}".format(perplexity))

Evaluate Perplexity:       1.00


In [74]:
#Save tokenizer and model
trainer._save("./arabert2arabert")
tokenizer.save_pretrained("./arabert2arabert")

Saving model checkpoint to ./arabert2arabert
Configuration saved in ./arabert2arabert/config.json
Model weights saved in ./arabert2arabert/pytorch_model.bin
tokenizer config file saved in ./arabert2arabert/tokenizer_config.json
Special tokens file saved in ./arabert2arabert/special_tokens_map.json


('./arabert2arabert/tokenizer_config.json',
 './arabert2arabert/special_tokens_map.json',
 './arabert2arabert/vocab.txt',
 './arabert2arabert/added_tokens.json',
 './arabert2arabert/tokenizer.json')

### Gradio Demo This allows you to create a sharable web application of the model


In [39]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.1 MB 28.6 MB/s 
     |████████████████████████████████| 54 kB 3.9 MB/s 
     |████████████████████████████████| 212 kB 71.8 MB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 84 kB 4.5 MB/s 
     |████████████████████████████████| 270 kB 75.0 MB/s 
     |████████████████████████████████| 2.3 MB 56.5 MB/s 
     |████████████████████████████████| 112 kB 60.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 68 kB 5.7 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 4.1 MB 45.7 MB/s 
     |████████████████████████████████| 594 kB 50

In [75]:
from transformers import EncoderDecoderModel, AutoTokenizer
import gradio as gr
from datasets import load_dataset
from torch.utils.data.dataloader import DataLoader
from transformers import default_data_collator
from torch.utils.data.sampler import SequentialSampler
import torch
from tqdm.notebook import tqdm

In [102]:
from arabert.preprocess import ArabertPreprocessor

model_name = "bert-base-arabert"
arabert_prep = ArabertPreprocessor(model_name=model_name, keep_emojis=False)

[2022-09-03 20:04:43,687 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./arabert2arabert")
model = EncoderDecoderModel.from_pretrained("./arabert2arabert")

model.to("cuda")
model.eval()
print("done")

In [ ]:
def generate_response(text, minimum_length, k):
  text_clean = arabert_prep.preprocess(text)
  inputs = tokenizer.encode_plus(text_clean,return_tensors='pt')
  outputs = model.generate(input_ids = inputs.input_ids.to("cuda"),
                   attention_mask = inputs.attention_mask.to("cuda"),
                   num_beams=1,
                   do_sample = True,
                   min_length=minimum_length,
                   top_k = k,
                   temperature = 1,
                   length_penalty =2)
  preds = tokenizer.batch_decode(outputs) 
  response = str(preds)
  response = response.replace("\'", '')
  response = response.replace("[[CLS]", '')
  response = response.replace("[SEP]]", '')
  response = str(arabert_prep.desegment(response))
  return response

In [45]:
gr.Interface(fn=generate_response,
             inputs=[
                 gr.inputs.Textbox(),
                 gr.inputs.Slider(5, 20, step=1, label='Minimum Output Length'),
                 gr.inputs.Slider(10, 1000, step=10, label='Top-K'),
                 ],
             outputs="text").launch(share=True)

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import yo

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://18286.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7ff667e8c810>,
 'http://127.0.0.1:7860/',
 'https://18286.gradio.app')